In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
#helper function to find lines in html document with DOI and titles of articles
def has_doi(href):
    return href and re.compile("chapter/").search(href)

In [4]:
#main mining function returning the initial dataframe 

def mining(html_doc, year, current_page, all_pages, part):
    #opening the html document (copy pasted and saved as a .doc file)
    doc = open(html_doc, "r", encoding = "utf-8") 
    soup = BeautifulSoup(doc, 'html.parser' )

    list_of_doi = soup.find_all(href=has_doi)
        
    #getting the titles and the doi's from list generated helper function

    titles = []
    doi_str = []

    for element in list_of_doi:
        titles.append(element.get_text()) #returns the titles as the only text in the list
        string = str(element)
        first_substring = '/chapter'
        second_substring ='">'
        #separates out the DOIS (added the +9 to remove /chapter/ from the beginning of all DOIS)
        doi_str.append(string[(string.find(first_substring)+9):string.find(second_substring)]) 
            
    ## now the lines containing author are found
    authors = soup.find_all("li", class_="c-author-list__item")
        
    #keeping only the author names
    authors_str = []
    for element in authors:
        string = str(element)
        first_substring = 'item">'
        second_substring ='</li>'
        authors_str.append(string[(string.find(first_substring)+6):string.find(second_substring)])
            
    #now the lines containing page numbers are found
    page_numbers= soup.find_all('div', class_ = "c-meta")
        
    #keeping only the page numbers
    page_numbers_str = []

    for element in page_numbers:
        string = element.get_text()[6:-1] #removes the white spaces and "Pages "
        both = string.split("-")
        if 'x' not in string: #filtering out front matters
            try: 
                if int(both[1])-int(both[0]) > 1: 
                    page_numbers_str.append(string)
            except:
                if "C" in string or "E" in string: #including corrections and erratum, are removed later
                    page_numbers_str.append(string)
    #filtering out back matters, only an issue in 2021 
    if year == 2021 and int(current_page) == int(all_pages):
        page_numbers_str = page_numbers_str[:-1]
                
            
    #need to create a list of the year of publication to add to dataframe 
    year_of_pub = []
    for element in titles:
        year_of_pub.append(year)
        
    #will add the part of the publication to the dataframe as well
    part_of_pub = []
    for element in titles:
        part_of_pub.append(part)
    
    #creating the column names and content for the dataframe        
    data = {'Title': titles,
        'Authors': authors_str,
        'Page numbers' : page_numbers_str,
        'DOI': doi_str,
        'Year of publication' : year_of_pub,
        'Part of publication' : part_of_pub
        }

    df = pd.DataFrame(data)

    return df 


In [5]:
#helper function to combine all df together
def data_together(data, year):
    combined_frame = pd.concat(data, ignore_index = True, sort = False)
    if year == 2012:
        combined_frame.drop(combined_frame[combined_frame["Title"].str.contains("Erratum")].index, inplace = True)
    elif year == 2021:
        combined_frame.drop(combined_frame[combined_frame["Title"].str.contains("Correction to")].index, inplace=True)
    combined_frame.to_csv('database_miccai_'+ str(year) +'.csv')
    return combined_frame

In [6]:
#saving all 2012 together as one with the helper function above
miccai =['miccai 2012 part 1 page 1 of 5.doc', 'miccai 2012 part 1 page 2 of 5.doc', 
         'miccai 2012 part 1 page 3 of 5.doc' , 'miccai 2012 part 1 page 4 of 5.doc', 
         'miccai 2012 part 1 page 5 of 5.doc', 
         
         'miccai 2012 part 2 page 1 of 5.doc', 
         'miccai 2012 part 2 page 2 of 5.doc', 'miccai 2012 part 2 page 3 of 5.doc' ,
         'miccai 2012 part 2 page 4 of 5.doc', 'miccai 2012 part 2 page 5 of 5.doc', 
         
         
         'miccai 2012 part 3 page 1 of 5.doc', 'miccai 2012 part 3 page 2 of 5.doc', 
         'miccai 2012 part 3 page 3 of 5.doc' , 'miccai 2012 part 3 page 4 of 5.doc', 
         'miccai 2012 part 3 page 5 of 5.doc']

data = []
for element in miccai:  
    data.append(mining(element, 2012, element[24:25], element[29:30], element[17:18]))

data_together(data, 2012)

,Title,Authors,Page numbers,DOI,Year of publication,Part of publication
0,Reliable Assessment of Perfusivity and Diffusi...,"M. Freiman, S. D. Voss, R. V. Mulkern, J. M. P...",1-9,10.1007/978-3-642-33415-3_1,2012.0,1
1,Multi-organ Abdominal CT Segmentation Using Hi...,"Robin Wolz, Chengwen Chu, Kazunari Misawa, Ken...",10-17,10.1007/978-3-642-33415-3_2,2012.0,1
2,Radiation-Free Drill Guidance in Interlocking ...,"Benoit Diotte, Pascal Fallavollita, Lejing Wan...",18-25,10.1007/978-3-642-33415-3_3,2012.0,1
3,Developing Essential Rigid-Flexible Outer Shea...,"Siyang Zuo, Takeshi Ohdaira, Kenta Kuwana, Yos...",26-33,10.1007/978-3-642-33415-3_4,2012.0,1
4,Surgical Gesture Classification from Video Data,"Benjamín Béjar Haro, Luca Zappella, René Vidal",34-41,10.1007/978-3-642-33415-3_5,2012.0,1
5,Remote Ultrasound Palpation for Robotic Interv...,"Caitlin Schneider, Ali Baghani, Robert Rohling...",42-49,10.1007/978-3-642-33415-3_6,2012.0,1
6,Modeling and Real-Time Simulation of a Vascula...,"Igor Peterlík, Christian Duriez, Stéphane Cotin",50-57,10.1007/978-3-642-33415-3_7,2012.0,1
7,Efficient Optic Cup Detection from Intra-image...,"Yanwu Xu, Jiang Liu, Stephen Lin, Dong Xu, Car...",58-65,10.1007/978-3-642-33415-3_8,2012.0,1
8,Population-Based Design of Mandibular Plates B...,"Habib Bousleiman, Christof Seiler, Tateyuki Ii...",66-73,10.1007/978-3-642-33415-3_9,2012.0,1
9,Thoracic Abnormality Detection with Data Adapt...,"Yang Song, Weidong Cai, Yun Zhou, Dagan Feng",74-81,10.1007/978-3-642-33415-3_10,2012.0,1


In [7]:
#saving all 2021 together as one with the helper function above
miccai =['miccai 2021 part 1 page 1 of 4.doc', 'miccai 2021 part 1 page 2 of 4.doc', 
         'miccai 2021 part 1 page 3 of 4.doc' , 'miccai 2021 part 1 page 4 of 4.doc', 
         
         'miccai 2021 part 2 page 1 of 4.doc', 'miccai 2021 part 2 page 2 of 4.doc', 
         'miccai 2021 part 2 page 3 of 4.doc' , 'miccai 2021 part 2 page 4 of 4.doc',
         
         'miccai 2021 part 3 page 1 of 4.doc', 'miccai 2021 part 3 page 2 of 4.doc', 
         'miccai 2021 part 3 page 3 of 4.doc' , 'miccai 2021 part 3 page 4 of 4.doc',
         
        'miccai 2021 part 4 page 1 of 4.doc', 'miccai 2021 part 4 page 2 of 4.doc', 
         'miccai 2021 part 4 page 3 of 4.doc' , 'miccai 2021 part 4 page 4 of 4.doc',
         
        'miccai 2021 part 5 page 1 of 5.doc', 'miccai 2021 part 5 page 2 of 5.doc', 
         'miccai 2021 part 5 page 3 of 5.doc' , 'miccai 2021 part 5 page 4 of 5.doc', 
         'miccai 2021 part 5 page 5 of 5.doc',
        
        'miccai 2021 part 6 page 1 of 4.doc', 'miccai 2021 part 6 page 2 of 4.doc', 
         'miccai 2021 part 6 page 3 of 4.doc' , 'miccai 2021 part 6 page 4 of 4.doc',
        
        'miccai 2021 part 7 page 1 of 5.doc', 'miccai 2021 part 7 page 2 of 5.doc', 
         'miccai 2021 part 7 page 3 of 5.doc' , 'miccai 2021 part 7 page 4 of 5.doc', 
         'miccai 2021 part 7 page 5 of 5.doc', 
        
        'miccai 2021 part 8 page 1 of 4.doc', 'miccai 2021 part 8 page 2 of 4.doc',
        'miccai 2021 part 8 page 3 of 4.doc', 'miccai 2021 part 8 page 4 of 4.doc']

data = []
for element in miccai:  
    data.append(mining(element, 2021, element[24:25], element[29:30], element[17:18]))
data_together(data, 2021)

,Title,Authors,Page numbers,DOI,Year of publication,Part of publication
0,Noisy Labels are Treasure: Mean-Teacher-Assist...,"Zhe Xu, Donghuan Lu, Yixin Wang, Jie Luo, Jaga...",3-13,10.1007/978-3-030-87193-2_1,2021,1
1,TransFuse: Fusing Transformers and CNNs for Me...,"Yundong Zhang, Huiye Liu, Qiang Hu",14-24,10.1007/978-3-030-87193-2_2,2021,1
2,Pancreas CT Segmentation by Predictive Phenoty...,"Yucheng Tang, Riqiang Gao, Hohin Lee, Qi Yang,...",25-35,10.1007/978-3-030-87193-2_3,2021,1
3,Medical Transformer: Gated Axial-Attention for...,"Jeya Maria Jose Valanarasu, Poojan Oza, Ilker ...",36-46,10.1007/978-3-030-87193-2_4,2021,1
4,Anatomy-Constrained Contrastive Learning for S...,"Bo Zhou, Chi Liu, James S. Duncan",47-56,10.1007/978-3-030-87193-2_5,2021,1
5,Study Group Learning: Improving Retinal Vessel...,"Yuqian Zhou, Hanchao Yu, Humphrey Shi",57-67,10.1007/978-3-030-87193-2_6,2021,1
6,Multi-phase Liver Tumor Segmentation with Spat...,"Yue Zhang, Chengtao Peng, Liying Peng, Huimin ...",68-77,10.1007/978-3-030-87193-2_7,2021,1
7,Convolution-Free Medical Image Segmentation Us...,"Davood Karimi, Serge Didenko Vasylechko, Ali G...",78-88,10.1007/978-3-030-87193-2_8,2021,1
8,Consistent Segmentation of Longitudinal Brain ...,"Jie Wei, Feng Shi, Zhiming Cui, Yongsheng Pan,...",89-98,10.1007/978-3-030-87193-2_9,2021,1
9,A Multi-branch Hybrid Transformer Network for ...,"Yinglin Zhang, Risa Higashita, Huazhu Fu, Yanw...",99-108,10.1007/978-3-030-87193-2_10,2021,1
